# Live plotting the derivative

This tutorial shows how to add a column-wise derivative to a 2D measurement

In [ ]:
# imports
import numpy as np
import qcodes as qc
from qcodes.utils.wrappers import init, do1d, do2d
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.instrument.parameter import ArrayParameter

## Setting up a mock experiment

In [ ]:
# Dummy instruments

dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
lockin = DummyInstrument('lockin', gates=['X', 'Y'])

station = qc.Station(dac, lockin)

init('./sandboxdata', 'sandboxsample', station, annotate_image=False)

In [ ]:
# add a mock non-trivial signal array parameter to the lock-in

class Signal(ArrayParameter):
    
    def __init__(self, name, instrument):
        super().__init__(name,
                         shape=(500,),
                         unit='arb. un.',
                         setpoint_names=('Voltage',),
                         setpoint_units=('V',)
                        )
        self.setpoints = (tuple(np.linspace(-3, 3, 500)),)
        self._instrument = instrument
        self.xpoint = self._xpoint()
    
    def reset_signal(self):
        self.xpoint = self._xpoint()
    
    
    def _xpoint(self):
        """
        A frequency counter
        """
        n = 0
        xx = np.linspace(-3, 3, 50)
        while True:
            yield xx[(n % len(xx))]
            n += 1
    
    
    def get(self):
        yy = np.array(self.setpoints[0])
        x = next(self.xpoint)
        sig = (1 - x/2 + x**5 + yy**3) * np.exp(-x**2 - yy**2)
        noise =  0.01*np.random.randn(500)
        sig += np.convolve(noise, np.hanning(5), mode='same')/np.sum(np.hanning(5))
        return sig
    
lockin.add_parameter(name='signal',
                     parameter_class=Signal)
        

In [ ]:
# Measure the beautiful signal

do1d(dac.ch1, 0, 1, 50, 0.02, lockin.signal)

## Adding the derivative

In [ ]:
# Define a new parameter. Since this will return an array of values, it must
# be a subclass of the ArrayParameter

class Derivative(ArrayParameter):
    
    def __init__(self, name, instrument, antiderivative):
        """
        The antiderivative is the parameter we wish to differentiate
        """
        super().__init__(name, 
                         shape=(antiderivative.shape[0] - 1,),  # derivative is one shorter
                         setpoint_names=antiderivative.setpoint_names,
                         setpoint_units=antiderivative.setpoint_units)
        self._instrument = instrument
        self.ad = antiderivative
        self.setpoints = (self.ad.setpoints[0][:-1],)
        
    def get(self):
        yy = self.ad.get_latest()
        xx = np.array(self.ad.setpoints[0])
        
        return np.diff(yy)/np.diff(xx)
    

In [ ]:
lockin.add_parameter('deriv',
                     antiderivative=lockin.signal,
                     parameter_class=Derivative)

In [ ]:
do1d(dac.ch1, 0, 1, 50, 0.2, lockin.signal, lockin.deriv, use_threads=False)

## Adding the derivative + smoothing

Directly taking the derivative of experimental data with noise usually results in a very noisy derivative signal.
Here we add some pre-smoothening of the signal prior to taking the derivative.

In [ ]:
class SmoothDerivative(ArrayParameter):
    
    def __init__(self, name, instrument, antiderivative, kernel_size):
        """
        The antiderivative is the parameter we wish to differentiate
        Some pre-smoothing is added
        """
        super().__init__(name, 
                         shape=(antiderivative.shape[0] - 1 - kernel_size,),
                         setpoint_names=antiderivative.setpoint_names,
                         setpoint_units=antiderivative.setpoint_units)
        self._instrument = instrument
        self.ad = antiderivative
        self.ks = kernel_size
        self.setpoints = (tuple(np.array(self.ad.setpoints[0])[self.ks//2:-self.ks//2-1]),)
    
    def change_kernel_size(self, ks):
        """
        Update the kernel size for more/less agressive smoothening
        """
        
        if (ks % 2) != 0:
            raise ValueError('Kernel size must be an even integer')
        
        self.ks = ks
        self.setpoints = (tuple(np.array(self.ad.setpoints[0])[self.ks//2:-self.ks//2-1]),)
        self.shape = (self.ad.shape[0] - 1 - self.ks,)
    
    @staticmethod
    def smoothen(signal, xx, ks):
        """
        Smoothen a signal and reduce the x-axis accordingly
        """
        if (ks % 2) != 0:
            raise ValueError('Kernel size must be an even integer'
                            )
        smooth_sig = np.convolve(signal, np.hanning(ks), mode='same')/np.sum(np.hanning(ks))
        smooth_sig = smooth_sig[ks//2:-ks//2]
        xx = xx[ks//2:-ks//2]
    
        return xx, smooth_sig
    
    def get(self):
        yy = self.ad.get_latest()
        xx = np.array(self.ad.setpoints[0])
        
        sxx, syy = self.smoothen(yy, xx, self.ks)
        
        return np.diff(syy)/np.diff(sxx)

In [ ]:
lockin.add_parameter('smoothderiv',
                     antiderivative=lockin.signal,
                     parameter_class=SmoothDerivative,
                     kernel_size=10)

In [ ]:
do1d(dac.ch1, 0, 1, 50, 0.2, lockin.signal, lockin.smoothderiv, use_threads=False)

### Changing kernel size

You can change the kernel size easily:

In [ ]:
lockin.smoothderiv.change_kernel_size(24)

In [ ]:
do1d(dac.ch1, 0, 1, 50, 0.2, lockin.signal, lockin.smoothderiv, use_threads=False)